In [0]:
%run ../utils/common

In [0]:
catalog_name = 'udp_wcm_dev'

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory_sapcar_hourly
(
  hash_id BIGINT,
  store_id STRING,
  product_id STRING,
  calday DATE,
  calhour BIGINT,
  closing_stock_qty DECIMAL(38,9)
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
spark.sql(f"""
          DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory_sapcar_hourly WHERE calday = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS);
""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory_sapcar_hourly 
WITH
total AS (
  SELECT DISTINCT
    FARM_FINGERPRINT(CONCAT(
        IFNULL(CAST(IF(c.store_id IS NOT NULL AND a.calday >= c.cutoff_date, c.store_id, CAST(a.store_id AS STRING)) AS STRING), ""),
        IFNULL(CAST(product_id AS STRING), ""),
        IFNULL(CAST(calday AS STRING), "")
    )) hash_id,
    IF(c.store_id IS NOT NULL AND a.calday >= c.cutoff_date, c.store_id, CAST(a.store_id AS STRING)) store_id,
    CAST(product_id AS STRING) product_id,
    calday,
    CASE
      WHEN calhour < 7 THEN 6
      WHEN calhour >= 7 AND calhour < 11 THEN 10
      WHEN calhour >= 11 AND calhour < 17 THEN 16
      WHEN calhour >= 17 AND calhour < 21 THEN 20
    ELSE NULL END calhour,
    closing_stock_qty
  FROM
    {catalog_name}.udp_wcm_silver_sap_car.sapcar_full_inventory a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_franchise_config c ON CAST(a.store_id AS STRING) = c.ref_fc_store
  WHERE calday = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
)
SELECT
  hash_id,
  store_id,
  product_id,
  calday,
  calhour,
  CAST(SUM(closing_stock_qty) AS DECIMAL(38,9)) closing_stock_qty
FROM total
GROUP BY 1,2,3,4,5
;
""")